# Bootstrap, L=20

In [ ]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import Operator

from kraus_channels import KrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver

from quantum_tools import  resample
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, CorruptionMatrix
from quantum_circuits import integrable_circuit


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
def load_data(filename, n, seed, L):
    with open(filename, 'rb') as f:
        data = pkl.load(f)


    data = marginalize_counts(data, 0)

    targets = counts_to_probs(data)
    targets_spam = targets[:6**n]
    targets_map = targets[6**n:]

    np.random.seed(seed)
    
    circuit_target = integrable_circuit(n+1, L)
    unitary = Operator(circuit_target).data

    inputs_spam, _ = generate_pauliInput_circuits(n)
        
    inputs_map, circuit_list_map = (
                generate_pauli_circuits(n, None, N=5000-6**n)
        )
    
    return inputs_spam, targets_spam, inputs_map, targets_map, unitary

def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue()], sample_freq=100),
                )

    model.train(inputs = inputs,
                targets = targets,
                inputs_val = [inputs],
                targets_val = [targets],
                num_iter = num_iter,
                N = 500,
                verbose=verbose
                )
    
    return model

In [3]:
path = '../data/chaos_exp_reorder/5q/integrable_L=20/'
n = 4
d = 2**n
L = 20
bootstrap_samples = 10

for i in range(1, 6):

    spam_list = []
    model_list = []

    seed = 42 + i

    inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

    tf.random.set_seed(seed)
    for j in range(bootstrap_samples):
        targets_spam_bs = resample(targets_spam, 12000)
        targets_map_bs = resample(targets_map, 12000)

        spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
        spam_list.append(spam_model) 


        model = fit_model(inputs_map, 
                        targets_map_bs, 
                        spam_model,
                        num_iter = 3000, 
                        verbose=False)
        model_list.append(model)


    model_saver(spam_list, f'models/integrable_spam_L=20_bootstrap_seed{seed}.model')
    model_saver(model_list, f'models/integrable_model_L=20_bootstrap_seed{seed}.model')

[0.9910804686789416]
[0.9912040638207186]
[0.9912486386226838]
[0.9911707342781589]
[0.9911820653004412]
[0.9911512173549715]
[0.9910900937174657]
[0.9910792702236226]
[0.991170420415364]
[0.9911638556568315]
[0.9922394936016735]
[0.992320107758393]
[0.9923199170112489]
[0.9923319433184448]
[0.9923061483016477]
[0.9922666084319768]
[0.9923216824186674]
[0.9922739146295385]
[0.9923265820483288]
[0.992266188634725]
[0.991934152649596]
[0.9919611595622172]
[0.9920279315075232]
[0.9919893885410604]
[0.9918869046322701]
[0.9920327160357069]
[0.991898392767705]
[0.9919249181924563]
[0.9919509101502824]
[0.9919305408418808]
[0.9930694840543223]
[0.9929981665072544]
[0.9929730901963199]
[0.9930089344946188]
[0.993026762611886]
[0.9929663936683146]
[0.993049222454302]
[0.9930509848234467]
[0.9930671128653523]
[0.9929782309551707]
[0.989959372002905]
[0.9900217946016207]
[0.9899809344443293]
[0.9899505730693162]
[0.9899918204820805]
[0.9899216277164715]
[0.989920360252385]
[0.9899310974113559]
[